In [ ]:
# This code is run in WSL
import os
os.getcwd()

'/content'

In [ ]:
# This code is run in jupyter
import os
os.getcwd()

'/content'

## Load Data from Kaggle

In [3]:
from google.colab import userdata
import os

os.environ["KAG1GLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')


In [7]:
!kaggle datasets download -d kunshbhatia/delhi-air-quality-dataset

! unzip "delhi-air-quality-dataset.zip"

Dataset URL: https://www.kaggle.com/datasets/kunshbhatia/delhi-air-quality-dataset
License(s): DbCL-1.0
delhi-air-quality-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  delhi-air-quality-dataset.zip
  inflating: final_dataset.csv       


In [8]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [10]:
df = pd.read_csv("final_dataset.csv")
df.head()

,Date,Month,Year,Holidays_Count,Days,PM2.5,PM10,NO2,SO2,CO,Ozone,AQI
0,1,1,2021,0,5,408.80,442.42,160.61,12.95,2.77,43.19,462
1,2,1,2021,0,6,404.04,561.95,52.85,5.18,2.60,16.43,482
2,3,1,2021,1,7,225.07,239.04,170.95,10.93,1.40,44.29,263
3,4,1,2021,0,1,89.55,132.08,153.98,10.42,1.01,49.19,207
4,5,1,2021,0,2,54.06,55.54,122.66,9.70,0.64,48.88,149


# 📊 Delhi Air Quality Dataset — Column Descriptions

## 📅 Date-related Features
- **Date** → Day of the month (1–31).  
- **Month** → Month number (1–12).  
- **Year** → Year (e.g., 2021).  
- **Days** → Day of the week (likely 1=Monday, 7=Sunday).  
- **Holidays_Count** → Number of holidays that day (0, 1, …).  

👉 *On holidays/weekends, traffic & industrial emissions may differ → air quality changes.*

---

## 🌫️ Pollutant Measurements
These are pollutant concentrations in the atmosphere (typically in **µg/m³**, except **CO** which is in **mg/m³**).

- **PM2.5** → Particulate Matter ≤ 2.5 micrometers  
  - Very fine particles that penetrate deep into lungs.  
  - Major contributor to poor AQI.  

- **PM10** → Particulate Matter ≤ 10 micrometers  
  - Larger dust particles.  
  - Harmful but less severe than PM2.5.  

- **NO₂ (Nitrogen Dioxide)**  
  - Emitted from vehicles, power plants, industries.  
  - High levels cause respiratory issues.  

- **SO₂ (Sulphur Dioxide)**  
  - From burning coal & industrial processes.  
  - Forms secondary particulates, contributes to acid rain.  

- **CO (Carbon Monoxide)**  
  - Toxic gas from vehicles & incomplete combustion.  
  - Even small amounts harmful indoors/outdoors.  

- **Ozone (O₃)**  
  - Ground-level ozone (bad ozone).  
  - Forms when NOx + VOCs react with sunlight.  
  - Higher in summer, major component of smog.  

---

## 🌍 Air Quality Measure
- **AQI (Air Quality Index)**  
  - A single number summarizing pollution level.  
  - Calculated using breakpoints for each pollutant (the worst pollutant determines AQI).  
  - **Larger AQI → worse air quality.**  

### Typical AQI Categories (CPCB/US EPA)
- Good: 0–50  
- Satisfactory  
- Moderate  
- Poor  
- Very Poor  
- Severe  

---

## 🔑 Takeaway
- **Predictors (X)** → Pollutants (PM2.5, PM10, NO₂, SO₂, CO, Ozone), plus **holiday/week/day features**.  
- **Target (y)** → AQI (numeric) or **AQI Category** (derived).  


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Date,1461.0,15.729637,8.803105,1.00,8.00,16.00,23.00,31.00
Month,1461.0,6.522930,3.449884,1.00,4.00,7.00,10.00,12.00
Year,1461.0,2022.501027,1.118723,2021.00,2022.00,2023.00,2024.00,2024.00
Holidays_Count,1461.0,0.189596,0.392116,0.00,0.00,0.00,0.00,1.00
Days,1461.0,4.000684,2.001883,1.00,2.00,4.00,6.00,7.00
PM2.5,1461.0,90.774538,71.650579,0.05,41.28,72.06,118.50,1000.00
PM10,1461.0,218.219261,129.297734,9.69,115.11,199.80,297.75,1000.00
NO2,1461.0,37.184921,35.225327,2.16,17.28,30.49,45.01,433.98
SO2,1461.0,20.104921,16.543659,1.21,7.71,15.43,26.62,113.40
CO,1461.0,1.025832,0.608305,0.27,0.61,0.85,1.24,4.70


## 📊 Analysis of `describe()` for PM2.5

### 🔹 Average
- On an average day, the air contains **90.77 µg/m³ of PM2.5**.  

---

### 🔹 Standard Deviation (SD)
- SD is used to tell how spread the data is from the mean.  
- **Why SD and not variance?** → because the units for mean and standard deviation are the same.  
  - This matters because we can compare it directly.  
  - Example: *“The average variation is ~72 µg/m³, which is meaningful since it’s in the same units as PM2.5 itself.”*  

- For a normal distribution only:  
  - 68% of values are between **−1σ and +1σ**.  
  - 95% of values are between **−2σ and +2σ**.  
  - 99.7% of values are between **−3σ and +3σ**.  

- But it looks like there is some **skewness** in the data → we will confirm with a boxplot.  

Here SD = **71.65**.  

- We can't comprend much about spread only from SD we will use CV for that.

---

### 🔹 Coefficient of Variation (CV)
- Formula: `CV = SD / Mean`.  
- Interpretation:  
  - CV < 0.1 (10%) → data is consistent, not widely spread.  
  - CV = 10–20% → low to moderate variability.  
  - CV = 20–50% → moderate to high variability.  
  - CV > 0.5 (50%) → data is widely spread, values fluctuate a lot.  

- Low CV → consistent and predictable.  
- High CV → inconsistent and unpredictable.  

👉 CV = 71.65 / 90.77 ≈ **0.79 (>0.5)** → data is widely spread, PM2.5 is inconsistent and varies a lot day to day.  

---

### 🔹 Range
- Range tells you spread at the extremes → how far apart the smallest and largest values are.

- But it doesn’t say anything about typical spread → that’s where SD and IQR come in.

- Range = Max − Min.  
- Range = 1000 − 0.5 ≈ **1000**.  
- SD = 72 → ~7% of the range, which looks small.  

⚠️ But the maximum (1000) is quite large and may be an outlier.  
So we cannot fully trust the range to judge spread.  

👉 Note: Even though SD is only ~7% of the full range, CV is 79%.  
This means the data is inconsistent and unpredictable → pollution varies a lot relative to typical levels.  
The “7%” is misleading because the max value (1000) is an extreme outlier.  

---

### 🔹 Skewness
- Mean >> Median → data is **right-skewed** (outliers pulling up the average).  
- Vice versa → left-skewed.  

👉 For PM2.5:  
- Median (50%) = **72.06**  
- Mean = **90.77**  
- Since mean > median, it looks **right-skewed** → we will reconfirm with a plot.  

---

### 🔹 Quartiles and IQR
- Quartiles represent % of data below a value. Example: 50% of data is below median (72.06).  

- Formula: `IQR = Q3 − Q1`  
  - Q1 (25%) = **41**  
  - Q3 (75%) = **118**  
  - IQR = 118 − 41 = **77**  

- Outlier bounds:  
  - Lower bound = Q1 − 1.5×IQR  
  - Upper bound = Q3 + 1.5×IQR  
  - Upper bound = 233.5  
  - But maximum = **1000** → clearly we have **outliers**.  

## Note

- **Range** = tells me extremes exist.
   - Range = 1000 → extreme span (but inflated by outlier).

- **SD** = tells me day-to-day values fluctuate a lot.
  - SD = 72 → typical deviation from mean is almost as large as the mean (unstable).

- **IQR** = tells me even typical days are spread out.
  - IQR = 77 → middle 50% of days range between 41–118 (still wide).

---

## ✅ Conclusion on PM2.5
- The data is **inconsistent and unpredictable**.  
- It is **widely spread**.  
- There are **potential outliers** (like max = 1000).  


## 📊 Analysis of `describe()` for PM10

### 🔹 Average
- On an average day, the air contains **218.22 µg/m³ of PM10**.  

---

### 🔹 Standard Deviation (SD)
- SD = **129.30**  
- Since SD is large compared to the mean, this shows **considerable spread**.

### 🔹Empirical Rule Check
- 1σ range → 89–347 (covers ~68% days if data was normal).

- 2σ range → −40–476 → practically 0–476.

- Max = 1000 → far beyond 2σ → confirms extreme outliers.

---

### 🔹 Coefficient of Variation (CV)
👉 CV = 129.3 / 218.22 ≈ **0.59 (>0.5)** → the data is **widely spread**, PM10 is inconsistent and varies a lot day to day.  

---

### 🔹 Range
- Range = 1000 − 9.69 ≈ **990.3 (~1000)**.  
- SD = 129.3 → ~13% of the range, which looks small.  

⚠️ But this is misleading because the max (1000) is an **outlier**.  
So we can’t fully rely on the range to comment on spread.  

---

### 🔹 Skewness
- Median (50%) = **199.8 (~200)**  
- Mean = **218.2**  
- Mean is slightly greater than median → suggests **mild right skew**.  
- Data is closer to normal than PM2.5, but still not perfectly symmetric.  

---

### 🔹 Quartiles and IQR
- Formula: `IQR = Q3 − Q1`  
  - Q1 (25%) = **115**  
  - Q3 (75%) = **298**  
  - IQR = **183**  

- Outlier bounds:  
  - Lower bound = Q1 − 1.5×IQR = **−159.5 (≈ 0)**  
  - Upper bound = Q3 + 1.5×IQR = **572.5**  
  - But maximum = **1000** → clearly we have **outliers**.

- Median = ~200, IQR = 183.

  - That means “middle 50%” of days already range widely (115–298).
  - 👉 Not just a few outliers — even the typical days fluctuate a lot.

## Range, SD, and IQR (for PM10)

- **Range** → shows the extremes.  
  - Min = 9.7, Max = 1000 → Range ≈ **990.3**.  
  - Tells us how far apart the smallest and largest values are (but inflated by outliers).  

- **Standard Deviation (SD)** → shows average spread around the mean.  
  - Mean = 218, SD = 129.  
  - This means a typical day differs from the mean by ~129 µg/m³, which is quite large.  

- **Interquartile Range (IQR)** → shows spread of the middle 50% of values.  
  - Q1 = 115, Q3 = 298 → IQR = **183**.  
  - So half the days had PM10 between 115 and 298 µg/m³ → still a wide spread.  

👉 Together:  
- Range tells me the **extreme span**.  
- SD tells me **average variability around the mean**.  
- IQR tells me the **typical variability (middle 50%)**, which is also widely spread.  

---

## ✅ Conclusion on PM10
- The data is **inconsistent and unpredictable**.  
- It is **widely spread**.  
- There are **potential outliers** (like max = 1000).  
- Distribution looks **closer to normal** than PM2.5, but still shows mild right skew.  

## 🔄 Comparing PM2.5 and PM10

- **PM2.5 CV = 79%** → more unstable, higher day-to-day fluctuations.  
- **PM10 CV = 59%** → still unstable, but relatively less variable than PM2.5.  

👉 Interpretation:  
- PM2.5 is **more unpredictable** (bigger swings relative to its mean).  
- PM10 is **slightly more stable**, but its **average concentration (218 µg/m³)** is higher than PM2.5 (90 µg/m³).  
- So, **PM2.5 drives instability**, while **PM10 contributes more to absolute pollution load**.  --> PM10’s mean concentration is much higher than PM2.5, so it makes up the bulk of particulate matter in the air, even though PM2.5 is more unstable and dangerous per unit.

## 📊 Analysis of `describe()` for NO₂

### 🔹 Average
- On an average day, the air contains **37.2 µg/m³ of NO₂**.  

---

### 🔹 Standard Deviation (SD)
- SD = **35.2**  
- Interpretation: on a typical day, NO₂ levels differ from the mean by ~35 µg/m³.  
- Since SD is almost equal to the mean, the data is **very widely spread**.  

---

### 🔹Empirical Rule Check
- 1σ range →  2-72.4 (covers ~68% days if data was normal).

- 2σ range → −33.3-107.6 → practically 0–107.6 (covers ~95% days)

- Max = 433.92 → far beyond 2σ → confirms extreme outliers.

---

### 🔹 Coefficient of Variation (CV)
- CV = 35.2 / 37.2 ≈ **0.95 (>0.5)**.  
- This means the data is **highly variable and unpredictable**.  

---

### 🔹 Range
- Range = Max − Min = 433.98 − 2.16 ≈ **432**.  
- Interpretation: the smallest and largest NO₂ values differ by 432 µg/m³.  
- Shows the extremes, but this can be inflated by outliers.  

---

### Skewness

- Mean (37.2) > Median (30.5) → the data is **right-skewed**.  

---

### 🔹 Interquartile Range (IQR)

- Q1 = 17.3, Q3 = 45.0 → IQR = **27.7**.  
- Outlier bounds:  
  - Lower = −24.7 (≈ 0)  
  - Upper = 87.0  
  - Max = 434 → clear **outliers**.  

- Meaning: the middle 50% of days had NO₂ between 17–45 µg/m³.  
- Compared to the mean (37.2), this is **large (~75% of mean)** → data is **widely spread** even in typical values.  


### 🔹 Range, SD, and IQR for NO₂

- **Range** → Data has extremes (2.16 to 433.98, span ≈ 432).  
  - Shows the full stretch of values, but this is inflated by outliers.  

- **Standard Deviation (SD)** → 35.2.  
  - On a typical day, NO₂ differs from the mean by ~35.  
  - Since this is almost equal to the mean (37.2), the data is **widely spread and unstable**.  

- **Interquartile Range (IQR)** → 27.7 (Q1 = 17.3, Q3 = 45.0).  
  - The middle 50% of days fall between 17–45 µg/m³.  
  - Compared to the mean (37.2), this is **large (~75% of the mean)**.  
  - This shows that even the “typical” days are **widely spread**.  

####  **Why You Can Call It “Widely Spread” wrt IQR**
- If IQR is small relative to the mean, data is stable.

- If IQR is large relative to the mean, data is unstable.

- Here: IQR (27.7) is ~75% of mean (37.2) → that’s very wide.

**NOTE**: Range shows the extremes, SD shows day-to-day instability, and IQR confirms that even the middle 50% of NO₂ values are widely spread relative to the mean.  


---

## ✅ Conclusion on NO₂
- NO₂ levels are **widely spread and unpredictable** (CV = 95%).  
- The **middle 50% of values** range from 17 to 45, but extremes go up to 434.  
- Suggests occasional **spikes or outliers** driving the high variability.  


In [16]:
df['PM2.5']

,PM2.5
0,408.80
1,404.04
2,225.07
3,89.55
4,54.06
...,...
1456,58.43
1457,33.83
1458,31.21
1459,38.01


In [12]:
df.shape

(1461, 12)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            1461 non-null   int64  
 1   Month           1461 non-null   int64  
 2   Year            1461 non-null   int64  
 3   Holidays_Count  1461 non-null   int64  
 4   Days            1461 non-null   int64  
 5   PM2.5           1461 non-null   float64
 6   PM10            1461 non-null   float64
 7   NO2             1461 non-null   float64
 8   SO2             1461 non-null   float64
 9   CO              1461 non-null   float64
 10  Ozone           1461 non-null   float64
 11  AQI             1461 non-null   int64  
dtypes: float64(6), int64(6)
memory usage: 137.1 KB


In [ ]:
# there are no null values as the shape count and info show same number of rows